In [3]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
SAGEMAKER_S3_BUCKET="sagemaker-us-west-2-412356575323"

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
# Access the Hugging Face API token
hugging_face_api_token = os.getenv('HUGGING_FACE_API_TOKEN')

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel
environment = {
    "HF_API_TOKEN": hugging_face_api_token,
    "HF_MODEL_ID": "mistralai/Mistral-7B-Instruct-v0.2",
    "HF_MODEL_QUANTIZE": "bitsandbytes"
}

# create Hugging Face Model Class
#model_data=f"s3://{SAGEMAKER_S3_BUCKET}/huggingface-pytorch-training-2024-06-15-00-46-44-736/output/model.tar.gz",  # path to your trained SageMaker model
huggingface_model = HuggingFaceModel(
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.26",                           # Transformers version used
   pytorch_version="1.13",                                # PyTorch version used
   py_version='py39',
   env=environment
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge",
)

# example request: you always need to define "inputs"
data = {
   "inputs": "<s>[INST] What is apache airflow? [/INST]"
}

# request
predictor.predict(data)

In [15]:
result = predictor.predict(data)
print(result)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\u0027mistral\u0027"
}
". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2024-06-15-05-38-42-851 in account 412356575323 for more information.

In [16]:
predictor.delete_endpoint()